In [63]:
import speech_recognition as sr
import pyttsx3
import os
import subprocess

In [64]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [ ]:
def listen():
    """Capture voice input and convert it to text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source)
            return recognizer.recognize_google(audio).lower()
        except sr.UnknownValueError:
            return None
        except sr.RequestError as e:
            print(f"Error with the speech recognition service: {e}")
            return None

In [ ]:
def listen_for_duration(duration=5):
    """Listen for a fixed duration and return the transcribed text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print(f"Listening for {duration} seconds...")
        try:
            # Adjust for ambient noise and record for the specified duration
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.record(source, duration=duration)
            return recognizer.recognize_google(audio).lower()
        except sr.UnknownValueError:
            return None
        except sr.RequestError as e:
            print(f"Error with the speech recognition service: {e}")
            return None

In [66]:
def execute_command(command):
    try:
        if "open" in command:
            file_name = command.replace("open", "").strip()
            subprocess.call(["xdg-open", file_name])
        elif "install" in command:
            package = command.replace("install", "").strip()
            os.system(f"sudo apt install {package}")
        elif "monitor" in command:
            os.system("htop")
        else:
            speak("Command not recognized.")
    except Exception as e:
        speak(f"Error: {str(e)}")

In [ ]:
if __name__ == "__main__":
    speak("Hello! Say 'Hey Tommy' to activate me.")
    
    while True:
        print("Waiting for 'Hey Tommy'...")
        command = listen()
        
        if command and "hey tommy" in command:
            speak("I am listening. What would you like to say?")
            print("I am listening. What would you like to say?")
            
            # Listen for the user's input after the wake word
            user_input = listen()
            
            if user_input:
                print(f"You said: {user_input}")
                speak(f"You said: {user_input}")
            else:
                speak("I didn't catch that. Please try again.")

In [72]:
import os
import struct
import logging
import pyaudio
import pvporcupine
import speech_recognition as sr
import pyttsx3

class VoiceAssistant:
    def __init__(self, porcupine_key, wake_word="hey google"):
        """
        Initialize the voice assistant with wake word detection.
        
        :param porcupine_key: Picovoice access key for wake word detection
        :param wake_word: Wake word to trigger the assistant
        """
        # Configure logging
        logging.basicConfig(level=logging.INFO, 
                            format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)

        # Speech recognition setup
        self.recognizer = sr.Recognizer()
        self.recognizer.dynamic_energy_threshold = True

        # Text-to-speech setup
        self.speech_engine = pyttsx3.init()
        self.speech_engine.setProperty('rate', 150)  # Adjust speaking rate

        # Wake word detection setup
        try:
            self.porcupine = pvporcupine.create(
                access_key=porcupine_key, 
                keywords=[wake_word]
            )
        except Exception as e:
            self.logger.error(f"Failed to initialize Porcupine: {e}")
            raise

        # Audio stream setup
        self.pyaudio = pyaudio.PyAudio()
        self.audio_stream = self.pyaudio.open(
            rate=self.porcupine.sample_rate,
            channels=1,
            format=pyaudio.paInt16,
            input=True,
            frames_per_buffer=self.porcupine.frame_length
        )

    def speak(self, text):
        """
        Convert text to speech.
        
        :param text: Text to be spoken
        """
        try:
            self.speech_engine.say(text)
            self.speech_engine.runAndWait()
        except Exception as e:
            self.logger.error(f"Text-to-speech error: {e}")

    def listen(self):
        """
        Capture voice input and convert it to text.
        
        :return: Recognized text or None
        """
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            try:
                self.logger.info("Listening...")
                audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=5)
                return self.recognizer.recognize_google(audio).lower()
            except sr.UnknownValueError:
                self.logger.warning("Could not understand audio")
                return None
            except sr.RequestError as e:
                self.logger.error(f"Speech recognition service error: {e}")
                return None
            except Exception as e:
                self.logger.error(f"Unexpected listening error: {e}")
                return None

    def run(self):
        """
        Main assistant loop with wake word detection.
        """
        self.speak("Hello! Say 'Hey Google' to activate me.")
        self.logger.info("Assistant initialized. Waiting for wake word...")

        try:
            while True:
                # Read audio frame
                pcm = self.audio_stream.read(self.porcupine.frame_length)
                pcm_int16 = struct.unpack_from("h" * self.porcupine.frame_length, pcm)
                
                # Check for wake word
                keyword_index = self.porcupine.process(pcm_int16)
                
                if keyword_index >= 0:  # Wake word detected
                    self.logger.info("Wake word detected!")
                    self.speak("I am listening. Please say something.")
                    
                    # Listen to user's command
                    user_input = self.listen()
                    
                    if user_input:
                        self.logger.info(f"User said: {user_input}")
                        self.speak(f"You said: {user_input}")
                    else:
                        self.speak("I didn't catch that. Please try again.")

        except KeyboardInterrupt:
            self.logger.info("Assistant stopped by user.")
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
        finally:
            self.cleanup()

    def cleanup(self):
        """
        Properly close and release resources.
        """
        try:
            if hasattr(self, 'audio_stream'):
                self.audio_stream.stop_stream()
                self.audio_stream.close()
            if hasattr(self, 'pyaudio'):
                self.pyaudio.terminate()
            if hasattr(self, 'porcupine'):
                self.porcupine.delete()
        except Exception as e:
            self.logger.error(f"Error during cleanup: {e}")

def main():
    # IMPORTANT: Replace with your actual Picovoice access key
    PORCUPINE_ACCESS_KEY = os.getenv('PORCUPINE_ACCESS_KEY', "erVMi3xKesYLcjbzzy1NZSMdGAUX0mpFIq8h5vI5rc6p/TxHHEU0hw==")
    
    try:
        assistant = VoiceAssistant(PORCUPINE_ACCESS_KEY)
        assistant.run()
    except Exception as e:
        logging.error(f"Failed to start assistant: {e}")

if __name__ == "__main__":
    main()

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm